<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_058_tensorflow_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## tf.estimator

we will create a machine learning model using tf.estimator and evaluate its performance. The dataset is rather small (7700 samples), so we can do it all in-memory. We will also simply pass the raw data in as-is

In [1]:
!pip install datalab

In [2]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

1.13.1



Read data created in the previous chapter.

In [0]:
csv_columns = ['fare_amount','pickuplon','pickuplat','dropofflon','dropofflat','passengers','key']
features = csv_columns[1:len(csv_columns)-1]
label = csv_columns[0]

df_train = pd.read_csv('./taxi-train.csv',header = None, names = csv_columns)
df_valid = pd.read_csv('./taxi-valid.csv',header = None, names = csv_columns)

### Input function to read from Pandas Dataframe into tf.constant

In [0]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[label],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000,
    num_threads =1
  )

### Create feature columns for estimator

In [0]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in features]
  return input_columns


### Linear Regression with tf.Estimator framework

In [17]:
tf.logging.set_verbosity(tf.logging.INFO)

outdir = 'taxi_trained'
shutil.rmtree(outdir, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = outdir)

model.train(input_fn = make_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f01c5ab5128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create C

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).